In [1]:
from multidoc2_chat_exp_init import *

In [2]:
inference_model = model_init()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.8.crossattention.self.query.bias', 'roberta.encoder.layer.7.crossattention.self.key.bias', 'roberta.encoder.layer.8.crossattention.output.LayerNorm.



Num Params: 277714524


In [3]:
state_dict = torch.load('trained_models/first_attempt_robert2gpt_/checkpoint-10726/pytorch_model.bin')
inference_model.load_state_dict(state_dict)

<All keys matched successfully>

In [4]:
inference_dataset = MultiDoc2ChatDataset(tokenizer,data_pack=[])
inference_dataset.change_data_mode(0)

In [23]:
chat_data = dict(facts="""If a value of 4 (dry) or more, or an increase of more than 1 (dry) from the previous month’s colour rating is obtained, a double membrane test should be performed as a first step in an investigation.""".replace('\n',''),
                 utterance="",
                 current_question="Hello!",
                 conv_history= ""
                 
                 )

input_data = inference_dataset.processExample(chat_data)


In [24]:
input_ids = pad_seq(input_data.input_ids,400,tokenizer.pad_token_id).view(1,-1).to(inference_model.device)
attention_mask = pad_seq(input_data.attention_mask,400,tokenizer.pad_token_id).view(1,-1).to(inference_model.device)
decoder_inputs = input_data.labels.view(1,-1).to(inference_model.device)

In [25]:
response = inference_model.generate(
    input_ids,
    attention_mask=attention_mask,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
    length_penalty=1.25,
    early_stopping=True,
    use_cache=True,
    num_beams=4,
    no_repeat_ngram_size=2,
    repetition_penalty=1.56,
)

In [26]:
tokenizer.decode(response[0])

'<s>[CONV_HISTORY] agent agent: Unfortunately, no relevant information is found. [SEP] user: No, I do not need to apply for a copy of my DD214 or other military service records. What if I need more information about the type of military records I will receive in order to obtain a DD-214? [RESPONSE] Do you need help with how to request a military skills test?</s>'